In [30]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [31]:
# load dataset
dataframe = pd.read_csv("Yes_Bank_Train.csv")
dataset = dataframe.values
dataframe =dataframe.drop(['serial number'],axis=1)
X = dataframe.drop(['credit_amount'],axis=1)
Y = dataframe['credit_amount']
obj_col=X.select_dtypes(include='object')
obj_col = obj_col.columns.values
X = pd.get_dummies(X, columns=obj_col)

In [32]:
# define the model
def larger_model():
# create model
    model = Sequential()
    model.add(Dense(120, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [33]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) RMSE" % (results.mean(), np.sqrt(results.std())))

Larger: -3231372.83 (1073.91) RMSE


In [15]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

In [21]:
# define base model
def baseline_model():
# create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [22]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [23]:
estimator.fit(X,Y)

In [24]:
df2 = pd.read_csv('Yes_Bank_Test_int.csv')
df2 =df2.drop(['serial number'],axis=1)
X_t = df2
obj_col=X_t.select_dtypes(include='object')
obj_col = obj_col.columns.values
X_t = pd.get_dummies(X_t, columns=obj_col)

In [25]:
finalpred = estimator.predict(X_t)

In [26]:
_ = pd.read_csv('Yes_Bank_Test_int.csv')
serial = _['serial number']
data = { 'serial number': serial, 'credit_amount': finalpred}
submission = pd.DataFrame(data)
submission.head()

,serial number,credit_amount
0,1,3814.772949
1,2,2732.864502
2,3,2994.148682
3,4,1337.371460
4,5,2208.149170


In [27]:
submission.to_csv('neural2.csv',index=False)

In [16]:
pipeline.fit(X,Y)

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f7666bc1cc0>)])

In [17]:
df2 = pd.read_csv('Yes_Bank_Test_int.csv')
df2 =df2.drop(['serial number'],axis=1)
X_t = df2
obj_col=X_t.select_dtypes(include='object')
obj_col = obj_col.columns.values
X_t = pd.get_dummies(X_t, columns=obj_col)

In [18]:
finalpred = pipeline.predict(X_t)

In [19]:
_ = pd.read_csv('Yes_Bank_Test_int.csv')
serial = _['serial number']
data = { 'serial number': serial, 'credit_amount': finalpred}
submission = pd.DataFrame(data)
submission.head()

,serial number,credit_amount
0,1,2509.455322
1,2,2102.790527
2,3,3340.221924
3,4,1512.605103
4,5,3716.088379


In [20]:
submission.to_csv('neural1.csv',index=False)

In [34]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Activation, Input, Add

In [49]:
hp = 0.2
inputs = Input(shape=[60])
x = Dense(1024, input_dim = [60])(inputs)
x = Dropout(hp)(x)

x_sh = x
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)

x = Dense(256)(x)
x_sh = x
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(128)(x)
x_sh = x
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#x = Dropout(hp)(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)
pre = Dense(1, activation='relu')(x)
model_4 = Model(input=inputs, output=pre)
model_4.compile(optimizer='Adadelta', loss='mean_squared_error')

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [50]:
model_4.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
dense_124 (Dense)               (None, 1024)         62464       input_5[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 1024)         0           dense_124[0][0]                  
__________________________________________________________________________________________________
dense_125 (Dense)               (None, 1024)         1049600     dropout_5[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

In [55]:
model_4.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=1024, epochs=1000)

Train on 640 samples, validate on 160 samples
Epoch 1/1000
640/640 [==============================] - 0s 284us/step - loss: 69213.4375 - val_loss: 5860532.0000
Epoch 2/1000
640/640 [==============================] - 0s 278us/step - loss: 73032.3984 - val_loss: 5973234.5000
Epoch 3/1000
640/640 [==============================] - 0s 271us/step - loss: 81569.0859 - val_loss: 5967213.0000
Epoch 4/1000
640/640 [==============================] - 0s 288us/step - loss: 84561.6016 - val_loss: 5896673.0000
Epoch 5/1000
640/640 [==============================] - 0s 290us/step - loss: 100295.3750 - val_loss: 6065029.0000
Epoch 6/1000
640/640 [==============================] - 0s 287us/step - loss: 91205.0469 - val_loss: 5788709.5000
Epoch 7/1000
640/640 [==============================] - 0s 274us/step - loss: 98544.7891 - val_loss: 6205123.5000
Epoch 8/1000
640/640 [==============================] - 0s 280us/step - loss: 108172.0000 - val_loss: 5913447.5000
Epoch 9/1000
640/640 [==================

Epoch 72/1000
640/640 [==============================] - 0s 263us/step - loss: 62210.4492 - val_loss: 5936437.0000
Epoch 73/1000
640/640 [==============================] - 0s 253us/step - loss: 57250.9883 - val_loss: 5892163.0000
Epoch 74/1000
640/640 [==============================] - 0s 285us/step - loss: 61041.4688 - val_loss: 5866715.5000
Epoch 75/1000
640/640 [==============================] - 0s 292us/step - loss: 62282.1133 - val_loss: 5861090.5000
Epoch 76/1000
640/640 [==============================] - 0s 306us/step - loss: 69942.2969 - val_loss: 5920700.0000
Epoch 77/1000
640/640 [==============================] - 0s 299us/step - loss: 61406.2891 - val_loss: 6054749.0000
Epoch 78/1000
640/640 [==============================] - 0s 288us/step - loss: 69910.3438 - val_loss: 5942538.5000
Epoch 79/1000
640/640 [==============================] - 0s 341us/step - loss: 61183.6172 - val_loss: 5951463.0000
Epoch 80/1000
640/640 [==============================] - 0s 289us/step - loss: 6

Epoch 143/1000
640/640 [==============================] - 0s 333us/step - loss: 54699.3242 - val_loss: 5929393.0000
Epoch 144/1000
640/640 [==============================] - 0s 277us/step - loss: 54404.5117 - val_loss: 6014757.5000
Epoch 145/1000
640/640 [==============================] - 0s 262us/step - loss: 57196.2305 - val_loss: 5862060.5000
Epoch 146/1000
640/640 [==============================] - 0s 265us/step - loss: 56219.4883 - val_loss: 5998829.0000
Epoch 147/1000
640/640 [==============================] - 0s 265us/step - loss: 54897.8984 - val_loss: 5894222.5000
Epoch 148/1000
640/640 [==============================] - 0s 259us/step - loss: 48594.5977 - val_loss: 5830626.5000
Epoch 149/1000
640/640 [==============================] - 0s 261us/step - loss: 45229.8203 - val_loss: 5932330.5000
Epoch 150/1000
640/640 [==============================] - 0s 259us/step - loss: 48921.4297 - val_loss: 5863593.0000
Epoch 151/1000
640/640 [==============================] - 0s 303us/step 

Epoch 214/1000
640/640 [==============================] - 0s 404us/step - loss: 55760.3984 - val_loss: 5825271.5000
Epoch 215/1000
640/640 [==============================] - 0s 352us/step - loss: 55167.6953 - val_loss: 5955977.0000
Epoch 216/1000
640/640 [==============================] - 0s 367us/step - loss: 44476.0625 - val_loss: 5792119.0000
Epoch 217/1000
640/640 [==============================] - 0s 364us/step - loss: 44666.1758 - val_loss: 5957496.5000
Epoch 218/1000
640/640 [==============================] - 0s 381us/step - loss: 42125.5234 - val_loss: 5822500.0000
Epoch 219/1000
640/640 [==============================] - 0s 358us/step - loss: 43143.7461 - val_loss: 5992216.0000
Epoch 220/1000
640/640 [==============================] - 0s 388us/step - loss: 44639.1797 - val_loss: 5869153.5000
Epoch 221/1000
640/640 [==============================] - 0s 420us/step - loss: 44663.3984 - val_loss: 5808998.5000
Epoch 222/1000
640/640 [==============================] - 0s 404us/step 

Epoch 285/1000
640/640 [==============================] - 0s 444us/step - loss: 42278.4375 - val_loss: 5849882.5000
Epoch 286/1000
640/640 [==============================] - 0s 264us/step - loss: 42451.3047 - val_loss: 5982451.0000
Epoch 287/1000
640/640 [==============================] - 0s 354us/step - loss: 42227.9766 - val_loss: 5914892.5000
Epoch 288/1000
640/640 [==============================] - 0s 359us/step - loss: 37182.9727 - val_loss: 5887269.0000
Epoch 289/1000
640/640 [==============================] - 0s 379us/step - loss: 43152.8789 - val_loss: 5908726.0000
Epoch 290/1000
640/640 [==============================] - 0s 400us/step - loss: 35684.4141 - val_loss: 5847019.5000
Epoch 291/1000
640/640 [==============================] - 0s 402us/step - loss: 43038.0820 - val_loss: 5927726.5000
Epoch 292/1000
640/640 [==============================] - 0s 422us/step - loss: 38918.1680 - val_loss: 5864289.0000
Epoch 293/1000
640/640 [==============================] - 0s 414us/step 

...

640/640 [==============================] - 0s 282us/step - loss: 116885.0625 - val_loss: 6036118.0000
Epoch 708/1000
640/640 [==============================] - 0s 285us/step - loss: 200763.2656 - val_loss: 6098267.0000
Epoch 709/1000
640/640 [==============================] - 0s 287us/step - loss: 134291.2188 - val_loss: 6060456.0000
Epoch 710/1000
640/640 [==============================] - 0s 270us/step - loss: 267880.0625 - val_loss: 6080193.0000
Epoch 711/1000
640/640 [==============================] - 0s 312us/step - loss: 101613.1719 - val_loss: 5961744.5000
Epoch 712/1000
640/640 [==============================] - 0s 303us/step - loss: 113235.2734 - val_loss: 6092959.5000
Epoch 713/1000
640/640 [==============================] - 0s 261us/step - loss: 200953.6719 - val_loss: 6160383.0000
Epoch 714/1000
640/640 [==============================] - 0s 272us/step - loss: 124422.0781 - val_loss: 6069672.0000
Epoch 715/1000
640/640 [==============================] - 0s 349us/step - loss:

Epoch 778/1000
640/640 [==============================] - 0s 258us/step - loss: 333171.2500 - val_loss: 6079942.5000
Epoch 779/1000
640/640 [==============================] - 0s 315us/step - loss: 154541.5469 - val_loss: 6017734.5000
Epoch 780/1000
640/640 [==============================] - 0s 367us/step - loss: 125205.7344 - val_loss: 6106480.5000
Epoch 781/1000
640/640 [==============================] - 0s 308us/step - loss: 260564.3438 - val_loss: 6019149.0000
Epoch 782/1000
640/640 [==============================] - 0s 253us/step - loss: 78850.9453 - val_loss: 5909514.0000
Epoch 783/1000
640/640 [==============================] - 0s 268us/step - loss: 80629.9766 - val_loss: 5994858.0000
Epoch 784/1000
640/640 [==============================] - 0s 270us/step - loss: 119365.4219 - val_loss: 6127945.0000
Epoch 785/1000
640/640 [==============================] - 0s 259us/step - loss: 197733.4375 - val_loss: 6347849.5000
Epoch 786/1000
640/640 [==============================] - 0s 258us

Epoch 849/1000
640/640 [==============================] - 0s 269us/step - loss: 64672.2734 - val_loss: 5902552.0000
Epoch 850/1000
640/640 [==============================] - 0s 259us/step - loss: 86747.9062 - val_loss: 6037695.0000
Epoch 851/1000
640/640 [==============================] - 0s 264us/step - loss: 147821.5781 - val_loss: 6206195.0000
Epoch 852/1000
640/640 [==============================] - 0s 264us/step - loss: 268885.6562 - val_loss: 6061144.0000
Epoch 853/1000
640/640 [==============================] - 0s 258us/step - loss: 82743.5156 - val_loss: 5879987.0000
Epoch 854/1000
640/640 [==============================] - 0s 269us/step - loss: 80374.5156 - val_loss: 6043040.0000
Epoch 855/1000
640/640 [==============================] - 0s 278us/step - loss: 154531.3594 - val_loss: 5974807.0000
Epoch 856/1000
640/640 [==============================] - 0s 257us/step - loss: 72595.3359 - val_loss: 5943641.0000
Epoch 857/1000
640/640 [==============================] - 0s 255us/st

Epoch 920/1000
640/640 [==============================] - 0s 262us/step - loss: 179779.3438 - val_loss: 5972631.0000
Epoch 921/1000
640/640 [==============================] - 0s 255us/step - loss: 63226.2891 - val_loss: 5830156.0000
Epoch 922/1000
640/640 [==============================] - 0s 254us/step - loss: 74155.7500 - val_loss: 5941872.0000
Epoch 923/1000
640/640 [==============================] - 0s 252us/step - loss: 126524.4766 - val_loss: 6218978.5000
Epoch 924/1000
640/640 [==============================] - 0s 281us/step - loss: 231580.0312 - val_loss: 6022741.0000
Epoch 925/1000
640/640 [==============================] - 0s 275us/step - loss: 67683.6719 - val_loss: 5831619.0000
Epoch 926/1000
640/640 [==============================] - 0s 258us/step - loss: 59484.4375 - val_loss: 5963761.5000
Epoch 927/1000
640/640 [==============================] - 0s 269us/step - loss: 54367.6680 - val_loss: 5855157.0000
Epoch 928/1000
640/640 [==============================] - 0s 269us/st

Epoch 991/1000
640/640 [==============================] - 0s 259us/step - loss: 159596.2500 - val_loss: 6114391.0000
Epoch 992/1000
640/640 [==============================] - 0s 257us/step - loss: 273713.5625 - val_loss: 6000631.0000
Epoch 993/1000
640/640 [==============================] - 0s 264us/step - loss: 78230.1406 - val_loss: 5912075.0000
Epoch 994/1000
640/640 [==============================] - 0s 252us/step - loss: 103826.0156 - val_loss: 6120789.0000
Epoch 995/1000
640/640 [==============================] - 0s 284us/step - loss: 201895.2188 - val_loss: 5910482.0000
Epoch 996/1000
640/640 [==============================] - 0s 257us/step - loss: 59166.0117 - val_loss: 5860129.0000
Epoch 997/1000
640/640 [==============================] - 0s 302us/step - loss: 80810.4141 - val_loss: 6025919.0000
Epoch 998/1000
640/640 [==============================] - 0s 309us/step - loss: 134577.1406 - val_loss: 6110468.0000
Epoch 999/1000
640/640 [==============================] - 0s 324us/

In [58]:
finalpred=model_4.predict(X_t)

In [66]:
    finalpred=finalpred.ravel()

In [67]:
_= pd.read_csv('Yes_Bank_Test_int.csv')
serial = _['serial number']
data = { 'serial number': serial, 'credit_amount': finalpred}
submission = pd.DataFrame(data)
submission.head()

,serial number,credit_amount
0,1,1490.946289
1,2,1172.130127
2,3,4065.040283
3,4,1725.057617
4,5,1185.279663


In [69]:
submission.to_csv('varanresnet.csv',index=False)

In [70]:
from neupy import algorithms, estimators, environment

ModuleNotFoundError: No module named 'neupy'